- Analyze individual microclimate data streams to compare and calculate similarity scores.
- Assess combined microclimate data streams for seasonal variations comparison.
- Group years based on microclimate similarities to identify patterns.
- Document all of my findings

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd

print('Done')

Done


In [7]:
#load the datasets
pecan = pd.read_csv()
malaria = pd.read_csv()


# We are calculating 2 different types of similarity metrics 
### 1.Similarity:
    - Pearson’s correlation
    - Spearman’s correlation
    - Kendall’s Tau
    - Cosine similarity
    - Jaccard similarity
### 2.Distance:
    - Euclidean distance
    - Manhattan distance

In [4]:
#Pearson's Correlation

#note to self, my intension is to writn them myself with numpy and then compary my results with the sklearn packages, otherwise it is to easy


In [ ]:
#Spearman's Correlation

In [ ]:
#Kendall's Tau

In [ ]:
#Cosine Similarity

In [ ]:
#Jaccard Similarity

In [ ]:
#Euclidean Distance

In [ ]:
#Manhatten Distance
